In [1]:
# https://developer.nytimes.com/apis
# https://developer.nytimes.com/docs/articlesearch-product/1/overview
# www.lucenetutorial.com/lucene-query-syntax.html

In [2]:
api_key='api-key=zBxlhvG69zXIH0BooIEibFf56eJpv5Ny'
api_site = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

In [23]:
#filter for politics
desk = "fq=news_desk:"
# pub_date: Timestamp (YYYY-MM-DD)
dates = "begin_date=20180101&end_date=20180331"
section = "fq=section_name:"

In [4]:
# Section Name Values
sectionNV = ['Today%27s%20Headlines','U.S.','Washington','Week in Review'] #list of Sections

# News Desk Values
newsDesk = ['Editorial','National','OpEd','Politics','U.S.','Washington'] #list of NewsDesks

In [6]:
import time
import urllib.request
import json

There are two rate limits per API: 4,000 requests per day and 10 requests per minute. You should sleep 6 seconds between calls to avoid hitting the per minute rate limit.

The Article Search API returns a max of 10 results at a time. 
The meta node in the response contains the total number of matches ("hits") and the current offset. 
Use the page query parameter to paginate thru results (page=0 for results 1-10, page=1 for 11-20, ...). 
You can paginate thru up to 100 pages (1,000 results). If you get too many results try filtering by date range.

page: must be less than or equal to 200

In [7]:
rateLimit = 0
# rate limit is per day!!!


In [24]:
print('Before: %s' % time.ctime())
arts = []

for i in newsDesk:
    url = api_site + desk + i + '&' + dates + '&sort=oldest' + '&' + api_key
    response = urllib.request.urlopen(url)
    rateLimit += 1
    articles = json.loads(response.read())
    
    print('Current NewsDesk',i,'has ',articles['response']['meta']['hits'],'articles')
    print(int(articles['response']['meta']['hits']//10)*6/60,'minutes until next newsDesk or max 20 mins',time.ctime())
    print('Current rate is: ',rateLimit)
    # Sleep 6 second to avoid "Exceeded Request Quota" error
    # 10 REQUESTS PER MINUTE
    time.sleep(6)
    
    for j in range(0,min(int(articles['response']['meta']['hits']//10),200)):
        # page limit of 200
        response = urllib.request.urlopen(url+"&page="+str(j))
        rateLimit += 1
        articles = json.loads(response.read())
        arts.extend(articles['response']['docs'])
                      
        # Sleep 6 second to avoid "Exceeded Request Quota" error
        # 10 REQUESTS PER MINUTE
        time.sleep(6)
        
     
        
print('After: %s\n' % time.ctime())

Before: Sun Mar 31 11:45:31 2019
Current NewsDesk Editorial has  180 articles
1.8 minutes until next newsDesk or max 20 mins Sun Mar 31 11:45:33 2019
Current rate is:  635
Current NewsDesk National has  538 articles
5.3 minutes until next newsDesk or max 20 mins Sun Mar 31 11:47:39 2019
Current rate is:  654
Current NewsDesk OpEd has  1197 articles
11.9 minutes until next newsDesk or max 20 mins Sun Mar 31 11:53:43 2019
Current rate is:  708
Current NewsDesk Politics has  18 articles
0.1 minutes until next newsDesk or max 20 mins Sun Mar 31 12:07:12 2019
Current rate is:  828
Current NewsDesk U.S. has  79 articles
0.7 minutes until next newsDesk or max 20 mins Sun Mar 31 12:07:25 2019
Current rate is:  830
Current NewsDesk Washington has  776 articles
7.7 minutes until next newsDesk or max 20 mins Sun Mar 31 12:08:19 2019
Current rate is:  838
After: Sun Mar 31 12:17:35 2019



In [25]:
'Articles for NewsDesk: {}'.format(len(arts))

'Articles for NewsDesk: 2750'

In [26]:
url+"&page="+str(j) # last url requested. Good for checking errors

'https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=news_desk:Washington&begin_date=20180101&end_date=20180331&sort=oldest&api-key=zBxlhvG69zXIH0BooIEibFf56eJpv5Ny&page=76'

In [27]:
print('Before: %s' % time.ctime())

for i in sectionNV:
    url = api_site + desk + i + '&' + dates + '&sort=oldest' + '&' + api_key
    response = urllib.request.urlopen(url)
    time.sleep(6)
    rateLimit += 1
    articles = json.loads(response.read())
    
    print('Current section',i,'has ',articles['response']['meta']['hits'],'articles')
    print(int(articles['response']['meta']['hits']//10)*6/60,'minutes until next section or max 20mins',time.ctime())
    print('Current rate is: ',rateLimit) 
    
    for j in range(0,min(int(articles['response']['meta']['hits']//10),200)):
        # page limit of 200
        response = urllib.request.urlopen(url+"&page="+str(j))
        rateLimit += 1
        articles = json.loads(response.read())
        arts.extend(articles['response']['docs'])
        
        # Sleep 6 second to avoid "Exceeded Request Quota" error
        # 10 REQUESTS PER MINUTE
        time.sleep(6)
        
print('After: %s\n' % time.ctime())

Before: Sun Mar 31 12:18:09 2019
Current section Today%27s%20Headlines has  567 articles
5.6 minutes until next section or max 20mins Sun Mar 31 12:18:18 2019
Current rate is:  916
Current section U.S. has  79 articles
0.7 minutes until next section or max 20mins Sun Mar 31 12:26:15 2019
Current rate is:  973
Current section Washington has  776 articles
7.7 minutes until next section or max 20mins Sun Mar 31 12:27:16 2019
Current rate is:  981
Current section Week in Review has  1650 articles
16.5 minutes until next section or max 20mins Sun Mar 31 12:38:14 2019
Current rate is:  1059
After: Sun Mar 31 13:03:37 2019



In [28]:
'Articles for NewsDesk and Section: {}'.format(len(arts))

'Articles for NewsDesk and Section: 5800'

In [29]:
news = []
# all articles

In [30]:
print('Before: %s' % time.ctime())

for i in range(0,len(arts)):
    dic = {}
    dic['id'] = arts[i]['_id']

    dic['headline'] = arts[i]['headline']['main']
    if ('news_desk' in arts[i]):
        dic['news_desk'] = arts[i]['news_desk']
    else:
        dic['news_desk'] = 'NA'
    dic['pub_date'] = arts[i]['pub_date'][0:10] # cutting time of day.
    if ('section_name' in arts[i]):
        dic['section'] = arts[i]['section_name']
    else:
        dic['section'] = 'NA'
    if arts[i]['snippet'] is not None:
        dic['snippet'] = arts[i]['snippet']
    else:
        dic['snippet'] = 'NA'
    if ('source' in arts[i]):
        dic['source'] = arts[i]['source']
    else:
        dic['source'] = 'NA'
    if ('type_of_material' in arts[i]):
        dic['type'] = arts[i]['type_of_material']
    else:
        dic['type'] = 'NA'
    dic['url'] = arts[i]['web_url']
    if ('word_count' in arts[i]):
        dic['word_count'] = arts[i]['word_count']
    # locations
    locations = []
    for x in range(0,len(arts[i]['keywords'])):
        if 'glocations' in arts[i]['keywords'][x]['name']:
            locations.append(arts[i]['keywords'][x]['value'])
    dic['locations'] = locations
    # subject
    subjects = []
    for x in range(0,len(arts[i]['keywords'])):
        if 'subject' in arts[i]['keywords'][x]['name']:
            subjects.append(arts[i]['keywords'][x]['value'])
    dic['subjects'] = subjects   
    news.append(dic)

print('After: %s\n' % time.ctime())    


Before: Sun Mar 31 14:25:33 2019
After: Sun Mar 31 14:25:48 2019



In [31]:
len(news)

5800

In [32]:
news[0]

{'headline': 'Capital Punishment Deserves a Quick Death',
 'id': '5c8866d740d36ff070b63e64',
 'locations': [],
 'news_desk': 'Editorial',
 'pub_date': '2018-01-01',
 'section': 'Opinion',
 'snippet': 'It’s time for the Supreme Court to end state-sanctioned killing for good.',
 'source': 'The New York Times',
 'subjects': ['Capital Punishment',
  'Eighth Amendment (US Constitution)',
  'Sentences (Criminal)'],
 'type': 'Editorial',
 'url': 'https://www.nytimes.com/2017/12/31/opinion/capital-punishment-death-penalty.html',
 'word_count': 1062}

In [33]:
'{} articles for period {}'.format(len(news),dates)
# number of articles

'5800 articles for period begin_date=20180101&end_date=20180331'

In [34]:
headers = ['id','headline','news_desk','pub_date','section','snippet','source','type','url','word_count','locations','subjects']
# column headers
headers

['id',
 'headline',
 'news_desk',
 'pub_date',
 'section',
 'snippet',
 'source',
 'type',
 'url',
 'word_count',
 'locations',
 'subjects']

# This will replace existing file. If you only want to add to the file, 
# skip this cell

In [35]:
file = open('articles.txt','w')

for i in headers:
    file.write(i)
    file.write('\t')
file.close()

#only do the first time to add headers

In [36]:
import re

In [37]:
file = open('articles.txt','a')

for i in range(0,len(news)):
    file.write('\n')
    for j in headers:        
        file.write(re.sub(r'[^\x00-\x7F]','',str(news[i][j]))) # remove unicode chars
        file.write('\t')
file.close()

# add articles to existing text file

In [38]:
len(news)

5800